# Initialization

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict

import xgboost as xgb
import seaborn as sns
import pandas as pd
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


sns.set_theme()
%matplotlib inline

# local development: remove this library
from google.colab import drive

In [ ]:
# local development: remove this line
drive.mount("/content/drive")

In [ ]:
# Define variables

# Excel filename for training purpose
# local development: path local
train_excel_file = "/content/drive/MyDrive/Freelance/PT Badak/dataset/after_turn_around.xlsx"
# Excel filename for testing purpose (Before Turn Around)
raw_excel_file = "/content/drive/MyDrive/Freelance/PT Badak/dataset/before_turn_around.xlsx"
# Randomness number
seed = 115
# Test size when training stage
test_size = 0.2 # 80% training 20% test
# Select the column that you want to use
# And rename it to make it easier for computer to read
# Make sure that the `selected_columns` and `normalized_column_names` should be
# the same size
selected_columns = ["TIMESTAMP", "MV", "PV", "AMINE>1C-2", "F/G > 1C-1", "1C-2 BOTTOM", "1C-4 > FG"]
normalized_column_names = ["timestamp", "mv", "pv", "amine_1c2", "fg_1c1", "bottom_1c2", "fg_1c4"]
# Row where the first row of the table is started
header = 4

# Define functions
def get_correlation_plot(
        df: pd.DataFrame,
        nrows: int,
        ncols: int,
        title: str,
        predictors: List[str],
        image_path: str = None):
    """
    Show scatter plot for each predictor against the target

    Parameters:
    df: Dataframe of your data
    nrows: how many plot in a row
    ncols: how many image in a column
    title: chart title for your plot
    predictors: list of predictor variable names
    image_path: location to store the plot
    """
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 10))
    fig.suptitle(title)
    i = 0
    max_len = len(predictors)
    for r in range(nrows):
        for c in range(ncols):
            if i >= max_len:
                break
            axes[r, c].scatter(x=df[predictors[i]], y=df["mv"]) # change mv if target is not `mv`
            axes[r, c].set_title(f"{predictors[i]} vs mv") # change mv if target is not `mv`
            i += 1
    if image_path:
        plt.savefig(image_path, dpi=300)

def get_kdeplot_mv(
        predictor_name: str,
        map_of_df: Dict[str, pd.DataFrame],
        title: str,
        image_path: str = None,
    ):
    """
    Show KDE plot for each predictor against the target

    Parameters:
    predictor_name: your predictor variable name
    map_of_df: mapping your dataframe with the predictor name
    title: chart title for your plot
    image_path: location to store the image
    """
    plt.figure(figsize=(16, 8))
    for k, v in map_of_df.items():
        sns.kdeplot(v[predictor_name], label=k)
    plt.title(title, fontdict={"size": 25})
    plt.legend()
    plt.tight_layout()
    if image_path:
        plt.savefig(image_path, dpi=300)

In [ ]:
# Read the data
df = pd.read_excel(train_excel_file, header=header)
df = df[selected_columns]
df.columns = normalized_column_names

# Show first 5 records
df.head()

In [ ]:
# Show descriptive statistics
df.describe()

# Exploratory Data Analysis (EDA)

In [ ]:
px.line(data_frame=df, x="timestamp", y="mv")

In [ ]:
# filter by date when the mv is dropping
mv_drop = df.loc[df["timestamp"] <= "2022-01-09"]
# filter by date when the mv is normal
mv_increase = df.loc[df["timestamp"] >= "2022-02-01"]
# filter by date when the mv is increasing
mv_normal = df.loc[(df["timestamp"] > "2022-01-09") & (df["timestamp"] < "2022-02-01")]

# mv df mapping
map_mv_df = {
    "mv_drop": mv_drop,
    "mv_increase": mv_increase,
    "mv_normal": mv_normal,
}

In [ ]:
for k, v in map_mv_df.items():
    get_correlation_plot(
        df=v,
        nrows=3,
        ncols=3,
        title=f"mv vs predictor when {k}",
        predictors=normalized_column_names[1:],
        image_path=f"mv_predictor_{k}.png",
    )

In [ ]:
for c in normalized_column_names[2:]:
    get_kdeplot_mv(
        predictor_name=c,
        map_of_df=map_mv_df,
        title=f"kdeplot for {c}",
        image_path=f"kdeplot_for_{c}.png",
    )

# Modelling

In [ ]:
# Split the data into training and validation dataset
X_train, X_test, y_train, y_test = train_test_split(
    X=df.drop(["mv", "timestamp"], axis=1),
    y=df["mv"],
    test_size=test_size,
    random_state=seed,
    shuffle=True,
)

In [ ]:
# Training pipeline, here the steps:
# 1. The training data will be normalized to be 0-1 range of values.
# 2. After normalisation, the data will be inputed to the regressor
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Predict the test data
xg_reg = xgb.XGBRegressor()
xg_reg.fit(X_train, y_train)
# Prediction
y_pred = xg_reg.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

In [ ]:
X_train

# Result

In [ ]:
raw = pd.ExcelFile(raw_excel_file)
raw.sheet_names

In [ ]:
df_raw = pd.concat([pd.read_excel(raw, sheet_name=s, header=header) for s in raw.sheet_names])
# df_raw = pd.read_excel(file_path, header=header)
df_raw = df_raw[selected_columns]
df_raw.columns = normalized_column_names
df_raw.info()

In [ ]:
# Get the raw data
X_before = scaler.fit_transform(df_raw.drop(["timestamp", "mv"], axis=1))
y_before = df_raw["mv"]

# Predict the raw data
y_pred_before = xg_reg.predict(X_before)

In [ ]:
final = df_raw[["timestamp", "mv"]]
final["mv_predicted"] = y_pred_before
final.reset_index(drop=True, inplace=True)

In [ ]:
june_july_data = final[
    (final["timestamp"] >= "2021-06-10") & \
    (final["timestamp"] <= "2021-07-10")] \
    .reset_index(drop=True) \
    .melt("timestamp", var_name="cols", value_name="vals")
june_july_data = june_july_data.drop_duplicates()

In [ ]:
px.line(x="timestamp", y="vals", color="cols", data_frame=june_july_data)